# Regressão Linear
> Encontrando a melhor linha

- toc: true 
- badges: true
- comments: true
- categories: [machine learning, aprendizado supervisionado]
<!-- - image: images/chart-preview.png -->

## TL;DR

Regressão linear é um modelo supervisionado de machine learning, que busca encontrar a linha que melhor representa um grupo de pontos.

---

# Senta que lá vem história...

Quando eu morava nos Estados Unidos, estudando engenharia mecânica, eu tive um curso sobre experiências - como as incertezas das medidas vinda dos aparelhos se apresentavam nos resultados. Não muito importante. Mas no experimento, eu e meu parceiro (vamos chamá-lo de Túlio) deveríamos pesar um [béquer](https://pt.wikipedia.org/wiki/B%C3%A9quer) com um líquido várias vezes (com diferentes volumes) e decidir se esse líquido era de água ou não - através do post, vamos supor sim.

Eu sabia que independentemente do que fosse, a gente deveria ver uma linha de pontos, já que todos líquidos têm uma densidade constante, e a peso total é proporcional ao volume $peso_{total} = densidade \cdot volume + peso_{béquer}$. Mas para a minha surpresa não foi bem isso que observamos.

<!-- <div style="text-align:center"><img src="" /></div> --> - manim: expectativa vs realidade

**Porquê?**

A balança não é exata, e quando medimos o volume também cometemos erros. Quando levamos tudo me consideração, isso faz diferença dependendo da sensibilidade do nosso experimento. Essas erros aleatórios que variam, normalmente são pequenos e não podemos prever chamamos de [ruído](https://pt.wikipedia.org/wiki/Ru%C3%ADdo). Mas mesmo com esse ruído todo, é possível encontrar os pontos se não tivéssemos ruído nenhum?

## O setup

Estamos tentando aproximar a peso de um béquer com água, sendo que sabemos a peso do béquer {% fn 2 %}. Ou seja:

* $P \leftarrow$ funcao do peso total do liquido no becker
* $\rho \leftarrow$ densidade do liquido
* $P_{becker} \leftarrow$ peso do becker

$$P(V) = \rho V + P_{becker}$$

> Tip: Fique atento às letras aqui e o que elas significam, porque vamos usá-las ao longo do post

Isso é o que queremos aproximar, mas também observamos o ruído. Com o ruído, a equação pode ser escrita assim:

* $\epsilon \leftarrow$ ruído (um pequeno erro e aleatório)

$$P(V) = \rho V + P_{becker} + \epsilon$$

## Abordagem ingênua

Quando olhamos, fica claro que os pontos formam uma nuvem ao redor de uma reta. E também não é muito dificil de traçar uma linha na mão. Linha traçada e problema resolvido.

A primeira coisa que eu pensei foi: vou no olho! Quão difícil pode ser?

<!-- <div style="text-align:center"><img src="" width="30%"/></div> --> -manim: draw line

<div style="text-align:center"><img src="https://media.giphy.com/media/3ztiZa4eICWGs/giphy.gif" width="30%"/></div>

Mas era bom demais pra ser verdade - o Túlio resolveu ele traçar uma linha também. Que era muito parecida com a minha, mas ele insistia que a dele era uma aproximação melhor. Como que a gente poderia comparar as linhas? Qual era a melhor? Como eu poderia provar pra ele que a **minha** era melhor?

## A função do erro

Nós estamos tentando aproximar a linha que melhor representa os pontos, certo? E, pra cada volume que medimos nós temos um peso observado e o peso que a nossa linha (nossa função) estimava. A diferença entre os dois nos da o erro pra esse ponto! Se somarmos todos os pontos, temos um erro da nossa linha! Ou seja, quem tiver a menor soma de erros tem a melhor linha. Visualmente fica:

<!-- <div style="text-align:center"><img src="" /></div> --> - manim: erro vertical

Mas também temos que tomar cuidado já que temos erros positivos e negativos - se o nosso ponto está acima ou abaixo da linha. Uma maneira fácil de resolver isso é tirar o quadrado desses erros, já que qualquer número (real) ao quadrado vai gerar um outro número positivo. Também sugeri dividirmos pelo número de pontos, pra poder comparar com as aproximações de outros grupos (que coletaram um número diferente de observações).

> Note: Nós não precisaríamos dividir pelo número de pontos já para mim e para o Túlio esse número é igual, e estamos vendo quem tem o **menor** erro entre nós.

<!-- <div style="text-align:center"><img src="" /></div> --> - manim: mse inteiro

Na verdade, não estávamos sendo nem um pouco inovativos. Essa métrica para comparar erros de valores numéricos existe e há bastante tempo. Em estatística, é muito comum ver isso como o [erro quadrático médio](https://pt.wikipedia.org/wiki/Erro_quadr%C3%A1tico_m%C3%A9dio) (ou MSE - "mean squared error", em inglês). Esse valor nos dá o quão bom uma função (no nosso caso uma linha) aproxima um grupo de pontos numéricos.

## Como melhorar o nosso chute?

Dúvida resolvida, e os números claramente mostravam que a minha aproximação tinha um erro menor. Mas depois disso, fiquei pensando: "se nós temos um erro da nossa função, não daria pra encontrar o mínimo dessa função e assim encontrar a menor linha possível?"

Sim. Podemos! Existem um grupo de algorítmos chamados de [algorítmos de otimização](https://pt.wikipedia.org/wiki/Otimiza%C3%A7%C3%A3o), que buscam fazer exatamente isso - encontrar mínimos (ou máximos) de uma função. Um desses algorítmos é o [gradient descent](https://murilo-cunha.github.io/inteligencia-superficial/machine%20learning/algoritmos%20de%20otimiza%C3%A7%C3%A3o/aprendizado%20supervisionado/2020/04/12/grad_desc.html).

Vamos ver como é que fica.

### A nossa linha

Antes de falar como fazemos pra melhorar a nossa aproximação, vamos lembrar da definição da nossa linha:

* $P \leftarrow$ função do peso total do líquido no béquer
* $\rho \leftarrow$ densidade do líquido
* $P_{becker} \leftarrow$ peso do béquer

$$P(V) = \rho V + P_{béquer}$$

E, mais formalmente falando, o que eu e o Túlio estávamos procurando são os melhores valores de $\rho$ e $P_{becker}$, já que são esses valores que definem a reta. E a função do erro então fica:

* $E \leftarrow$ função do erro da nossa linha baseado nas nossas observações
* $V_{obs} \leftarrow$ peso total observado
* $P_{obs} \leftarrow$ peso total observado
* $N \leftarrow$ número de observações


$$E(P_{béquer},\rho) =  \frac{1}{N}\sum_{obs=1}^{N}{(P(V_{obs}) - P_{obs})^2}$$

Eu sei que é mais intimidador quando colocamos tudo de uma vez em uma equação. Mas lembra que essa equação está descrevendo nada mais é do que discutimos [acima](##A-função-do-erro). Tome seu tempo pra verificar que faz sentido o que está acontecendo aqui - estamos somando os "erros verticais" ao quadrado, e dividindo a soma pelo número total de pontos.

Agora, como podemos ajustar os valores de $\rho$ e $P_{becker}$ pra melhorar nossa aproximação?

### Aplicando o gradient descent

Tudo o que precisamos fazer para aplicar o "gradient descent" (descida do gradiente) é definir uma função diferenciável que descreve o **erro** da nossa aproximação.

![]({{ site.baseurl }}/assets/manim/videos/grad_desc/480p15/CostSteps.gif "Passo-a-passo visualizado")

Lembre-se que em gradient descent, nós reduzimos o erro dando um passo na direção oposta do gradiente - ou seja, na direção oposta da derivada em cada dimensão{% fn 1 %}. Como que fica o update de cada parâmetro?

Mas antes de mergulharmos nas letrinhas, queria lembrar que essa provavelmente vai ser a parte mais confusa, especialmente se essa é a primeira vez que você está vendo isso. Mas vamos com calma. Vamos definir alguns termos daqui a pouco, mas também vamos explicá-los um por um, e até resolver um exemplo. Até o fim desse post tudo vai ficar mais claro. E fique à vontade para ler, pensar e reler, até que você fique confortável.

> Warning: Matemática à frente

<!-- <div style="text-align:center"><img src="" /></div> --> - manim: transformacoes da equacao

Ou então, um passo a passo mais detalhado (que parece mais complicado do que realmente é):

As derivadas parciais da nossa função de erro:

> Note: Para calcular as derivadas parciais da nossa função de erro nós devemos usar a [regra da cadeia](https://pt.khanacademy.org/math/ap-calculus-ab/ab-differentiation-2-new/ab-3-1a/a/chain-rule-review).

$$\frac{\partial E}{\partial P_{becker}}(P_{becker},\rho) =  \frac{2}{N}\sum_{obs=1}^{N}{((P(V_{obs}) - P_{obs}) \cdot \frac{\partial P}{\partial P_{becker}}(V_{obs}))}$$

$$\frac{\partial E}{\partial \rho}(P_{becker},\rho) =  \frac{2}{N}\sum_{obs=1}^{N}{((P(V_{obs}) - P_{obs}) \cdot \frac{\partial P}{\partial \rho}(V_{obs})})$$

As [derivadas parciais](https://pt.khanacademy.org/math/multivariable-calculus/multivariable-derivatives/partial-derivative-and-gradient-articles/a/introduction-to-partial-derivatives) da nossa linha (função que estamos tentando aproximar os pontos):

> Note: A derivada parcial $\frac{\partial f}{\partial x}(x,y)$ é coeficiente da linha tangente a $f$ se $y$ fosse um número constante.

$$\frac{\partial P}{\partial P_{becker}}(V_{obs}) = 1$$

$$\frac{\partial P}{\partial \rho}(V_{obs}) = V_{obs}$$


O passo para cada dimensão:

* $\alpha \leftarrow$ a taxa de aprendizado

$$P_{becker} \leftarrow P_{becker} - \alpha \cdot \frac{\partial E}{\partial P_{becker}}(P_{becker},\rho)$$

$$\rho \leftarrow P_{becker} - \alpha \cdot \frac{\partial E}{\partial\rho}(P_{becker},\rho)$$

E colocando tudo junto:

$$P_{becker} \leftarrow P_{becker} - \alpha \cdot \frac{2}{N}\sum_{obs=1}^{N}{(P(V_{obs}) - P_{obs})} \cdot 1$$

$$\rho \leftarrow \rho - \alpha \cdot \frac{2}{N}\sum_{obs=1}^{N}{((P(V_{obs}) - P_{obs}) \cdot V_{obs}})$$

E é "só" isso!

<div style="text-align:center"><img src="https://media.giphy.com/media/Ni4cpi0uUkd6U/giphy.gif" width="30%"/></div>

> Tip: Não se sinta intimidado. Lembre-se que a única coisa que estamos fazendo aqui é reduzir o valor de uma função (função do erro). O porquê o algoritmo funciona, ou a intuição por trás do algoritmo não está no escopo desse post. Mas fique à vontade para ler o post sobre [gradient descent](todo_grad) onde mergulhamos mais a fundo.

Visualmente, ficaria mais ou menos assim:

<!-- <div style="text-align:center"><img src="https://media.giphy.com/media/Ni4cpi0uUkd6U/giphy.gif" width="30%"/></div> --> manim: points, noise, bad line, better line, best line! (with update on number of steps)

### Um exemplo

Eu sei que é meio confuso, então vamos resolver um exercício simples: vamos tentar aproximar uma linha com três pontos.

<div style="text-align:center"><img src="https://media.giphy.com/media/HBWbIuHvXI2Eo/giphy.gif" width="30%"/></div>

Nesse exemplo, a linha representaria a **densidade real** da água, enquanto os pontos seriam **experimentos realizados**, mas agora vamos usar os termos $x$ e $y$ para simplificar o problema de um jeito que seria mais fácil de generalizar para outros casos.

Vamos fingir que pra linha temos:

$$f(x) = y = 2 \cdot x + 1 $$

E para os pontos observados:

$$p_1 = (1, 2.5)$$
$$p_2 = (2, 5.5)$$
$$p_3 = (3, 6.5)$$

> Note: estamos procurando a linha que reduz o erro, e essa pode ser (e provavelmente seria) diferente da linha ideal, livre de ruído. Nesse exemplo o nosso erro é de sempre $±0.5$, o que não aconteceria na vida real. Por conta disso, o nosso chute vai se aproximar da linha ideal no nosso exemplo.

#### Chute inicial

Quando eu estava realizando os experimentos, nós tentamos dar um chute inicial que se aproximasse ao máximo da nossa função de verdade. Na prática não é assim que acontece. Um bom primeiro chute reduz o número de passos que vamos dar. Mas na prática, quando lidamos com problemas mais complexos, não sabemos exatamente o que seria um bom ou mal chute, então escolhemos valores aleatórios para os parâmetros nossa linha ($m$ e $b$) - que também podemos chamar de **pesos** da nossa função.

In [8]:
#hide_input
from typing import List
import numpy as np
import pandas as pd
import altair as alt

def y_linha(x_list: List[float], m: float, b: float) -> List:
    """Retorna os valores de y dado x seguindo comportamento linear."""
    return [m*x+b for x in x_list]

# lists
m_chute = -1
b_chute = 2
x = [1, 2, 3]

y_ideal = [3, 5, 7]
y_obs = [2.5, 5.5, 6.5]
y_chute = y_linha(x, m_chute, b_chute)

# dataframes
df_linhas = pd.DataFrame({
    'Y': y_ideal + y_chute,
    'X': x + x,
    'Linha': ['Ideal']*3 + ['Chute']*3
})

df_pontos = pd.DataFrame({
    'X': x,
    'Observado': y_obs,
    'Chute': y_chute
})

# plots
plt_linhas = alt.Chart(df_linhas).encode(
    x='X',
    y='Y',
    color=alt.Color('Linha', scale=alt.Scale(domain=['Ideal', 'Chute'], range=['orange', 'blue']))
)

plt_pontos = alt.Chart(df_pontos).encode(
    x='X',
    y='Y',
)

plt_diff = alt.Chart(df_pontos).encode(
    alt.X('X:Q'),
    alt.Y('Chute:Q'),
    alt.Y2('Ideal:Q')
)


alt.layer(
    plt_linhas.mark_line(),
    plt_diff.mark_rule(color='red').encode(alt.X('X'), alt.Y('Chute'), alt.Y2('Observado')),
    plt_pontos.mark_circle(color='orange', opacity=1, size=40).encode(x='X', y='Observado'),
    plt_pontos.mark_circle(color='blue', opacity=1, size=40).encode(x='X', y='Chute'),
).properties(title='Observações').interactive()

alt.LayerChart(...)

In [9]:
#hide
import numpy as np


def mse(y_pred: np.array, y_true: np.array) -> float:
    """Calcule o erro quadrado médio entre dois vetores."""
    return (np.square(y_pred - y_true)).mean()

def _delta(var: float, y_pred: np.array, y_true: np.array, x_obs: np.array, is_b: bool, alpha: float):
    """Calcule a diferença dos pesos. Ou seja, retorne a derivada do erro quadrático médio multiplicado pela gradiente.
    :param var: valor da variável que estamos querendo fazer o update (valor de m ou b)
    :param y_pred: numpy array dos valores previstos, do nosso chute
    :param y_true: numpy array dos pontos de y encontrados no experimento
    :param is_b: valor booleana que indica se estamos fazendo o update de b ou não
    :param alpha: a taxa de aprendizado
    :param n: número de pontos/experimentos realizados"""
    
    def _media(y_pred: np.array, y_true: np.array, x: np.array, is_b: bool):
        """Calcule o valor da soma que é parte da derivada."""
        if is_b:
            factor = 1
        else:
            factor = x
        
        return ((y_pred - y_true) * factor).mean()
    
    assert y_pred.shape == y_true.shape, "Número de previsões e observados diferentes."
    assert y_pred.ndim == y_true.ndim == 1, "Y devem ser vetores."
    
    media = _media(y_pred, y_true, x_obs, is_b)
    
    return alpha * 2 * media

# converta as listas para um numpy arrays
x = np.array(x)
y_obs = np.array(y_obs)
y_chute = np.array(y_chute)

mse_antes = mse(y_obs, y_chute)

m_update = m_chute - _delta(m_chute, y_chute, y_obs, x, False, 0.1)
b_update = b_chute - _delta(b_chute, y_chute, y_obs, x, True, 0.1)

y_update = y_linha(x, m_update, b_update)

mse_depois = mse(y_obs, y_update)

if mse_depois >= mse_antes:
    """Verifique que o erro quadrático médio está diminuindo."""
    raise ValueError('Os valores não estão melhorando 😞 ...')
else:
    print(f"O erro foi de {mse_antes:.3f} para {mse_depois:.3f}! 👏 \n")

print(f"m := {m_update}")
print(f"b := {b_update}")
print(f"MSE = {mse_depois}")

O erro foi de 29.583 para 1.159! 👏 

m := 1.3333333333333335
b := 2.966666666666667
MSE = 1.1585185185185194


Ou seja, o para o nosso chute inicial, escolhemos:

* $m = -1$
* $b = 2$

$\therefore f(x) = y = -1 \cdot x + 2$

As linhas vermelhas mostram o erro, e a gente consegue conferir que o erro quadrático médio é:

$$E(m,b) =  \frac{1}{N}\sum_{obs=1}^{N}{(f(x_{obs}) - y_{obs})^2}$$
$$\therefore E(2,1) =  \frac{1}{3}\sum_{obs=1}^{N}{((-1 \cdot x_{obs} + 2) - y_{obs})^2}$$
$$\therefore E(2,1) =  \frac{1}{3}((-1 \cdot 1 + 2) - 2.5)^2 + ((-1 \cdot 2 + 2) - 3.5)^2 + ((-1 \cdot 3 + 2) - 6.5)^2)$$
$$\therefore E(2,1) =  29.58$$

Uma maneira de intrepertar esse erro é a diferença média ao quadrado. Poderíamos tirar a raiz quadrada do erro para termos um valor mais interpretável, mas como você vai ver daqui a pouco, vamos tirar a derivada desse valor. Raízes complicam esse processo. Além do mais, independentemente ser ao quadrado ou não, estamos procurando o mínimo desse erro.

#### Melhorando o chute

Lembrando que os nossos são atualizados de acordo com o que discutimos acima:

$$m \leftarrow m - \alpha \cdot \frac{2}{N}\sum_{obs=1}^{N}{(f(x_{obs}) - y_{obs})} \cdot x_{obs}$$

$$b \leftarrow b - \alpha \cdot \frac{2}{N}\sum_{obs=1}^{N}{(f(x_{obs}) - y_{obs})} \cdot 1$$

Vamos escolher nossa taxa de aprendizado arbitrariamente ($\alpha = 0.1$). Então os nossos novos pesos ficam:

$$m \leftarrow -1 - 0.1 \cdot \frac{2}{3}((-1 \cdot 1 + 2) - 2.5) + ((-1 \cdot 2 + 2) - 3.5) + ((-1 \cdot 3 + 2) - 6.5)) \cdot -1$$

$$b \leftarrow 2 - 0.1 \cdot \frac{2}{3}((-1 \cdot 1 + 2) - 2.5) + ((-1 \cdot 2 + 2) - 3.5) + ((-1 \cdot 3 + 2) - 6.5)) \cdot 1$$

$$\therefore$$

$$m \leftarrow 1.33$$

$$b \leftarrow 2.97$$

<!--  gif--> -- manim

E com esses novos valores de $m$ e $b$ o nosso erro vai de **29.58** para **1.16** (fique à vontade pra verificar)!

<div style="text-align:center"><img src="https://media.giphy.com/media/lMVNl6XxTvXgs/giphy.gif" width="30%"/></div>

In [10]:
#hide_input
from typing import List
import numpy as np
import pandas as pd
import altair as alt

# converta os numpy arrays de volta para listas (por conveniencia)
y_chute = y_chute.tolist()
x = x.tolist()

# dataframes
df_linhas = pd.DataFrame({
    'Y': y_ideal + y_update + y_chute,
    'X': x + x + x,
    'Linha': ['Ideal']*3 + ['Update']*3 + ['Inicial']*3
})

df_pontos = pd.DataFrame({
    'X': x,
    'Observado': y_obs,
    'Update': y_update
})

# plots
plt_linhas = alt.Chart(df_linhas).encode(
    x='X',
    y='Y',
    color=alt.Color('Linha', scale=alt.Scale(domain=['Ideal', 'Update', 'Inicial'], range=['orange', 'blue', 'red']))
)

plt_pontos = alt.Chart(df_pontos).encode(
    x='X',
    y='Y',
)

plt_diff = alt.Chart(df_pontos).encode(
    alt.X('X:Q'),
    alt.Y('Update:Q'),
    alt.Y2('Ideal:Q')
)


alt.layer(
    plt_linhas.mark_line(),
    plt_diff.mark_rule(color='red').encode(alt.X('X'), alt.Y('Update'), alt.Y2('Observado')),
    plt_pontos.mark_circle(color='orange', opacity=1, size=40).encode(x='X', y='Observado'),
    plt_pontos.mark_circle(color='blue', opacity=1, size=40).encode(x='X', y='Update'),
).properties(title='Observações').interactive()

alt.LayerChart(...)

E o resto é só repetindo o mesmo processo! O próximo passo eu deixo para você resolver. Como ficaria os novos pesos $m$ e $b$? Qual seria o erro quadrático médio?

In [203]:
#hide_input
# obs - quando estamos treinando um modelo de regressão linear, estamos executamos o código abaixo várias vezes!
# A redução do erro em com as iterações é o que chamamos de "aprender"

# converta as listas para um numpy arrays
x = np.array(x)
y_obs = np.array(y_obs)
y_update = np.array(y_update)

mse_antes = mse(y_obs, y_update)

mse_init = mse(y_obs, y_chute)
m_update = m_update - _delta(m_update, y_update, y_obs, x, False, 0.1)
b_update = b_update - _delta(b_update, y_update, y_obs, x, True, 0.1)

y_update = y_linha(x, m_update, b_update)

mse_depois = mse(y_obs, y_update)

if mse_depois >= mse_antes:
    """Verifique que o erro quadrático médio está diminuindo."""
    raise ValueError('Os valores não estão melhorando 😞 ...')
else:
    print(f"O erro foi de {mse_antes:.3f} para {mse_depois:.3f}! 👏 \n")

print(f"m := {m_update}")
print(f"b := {b_update}")

O erro foi de 0.222 para 0.222! 👏 

m := 1.9918348303106774
b := 0.8518946832576453


#### Vetorizando tudo

Agora, nós poderíamos colocar tudo em vetores. Se a gente fizer tudo certo, os resultados não mudariam. Mas antes de fazer isso, porque vetorizar tudo? Com certeza é mais trabalhoso reescrever tudo em `Numpy` e repensar em como reescrever suas funções. Vale a pena esse esforço? Além do mais, para multiplicar dois vetores nós ainda temos que computar o mesmo número de operações matemáticas certo?

Sim, isso é verdade. Mas ainda assim, vale (e muito) o esforço.
* `Numpy` arrays são mais eficientes que listas em `Python`. Muitas das operações são implementadas em [C/C++/Cython](https://en.wikipedia.org/wiki/NumPy#The_ndarray_data_structure), o que aumenta a velocidade de execução, além de usar [ponteiros](https://pt.wikipedia.org/wiki/Ponteiro_(programa%C3%A7%C3%A3o)), etc.
* Além disso, uma multiplicação de matrizes, por exemplo, engloba várias operações matemáticas (adição/subtração dos elementos da matriz). Passando várias instruções de uma vez só possibilita o computador de paralelizar as operações ([SIMD](https://pt.wikipedia.org/wiki/SIMD)). Se eu e o Túlio tivéssemos de fazer uma multiplicação de matriz na mão, eu poderia pegar a primeira metade, e ele a segunda. Desse jeito, a gente calcularia na metade do tempo.

Espero que tenha te convencido. Em Machine Learning, essas operações se repetem tantas vezes que acabam fazendo muita diferença. Colocando tudo em vetores, o nosso exemplo fica assim:

$$\vec{x_{obs}} = \begin{bmatrix} 1 \\ 2 \\ 3 \end{bmatrix}$$

$$\vec{y} = b + m \cdot \begin{bmatrix} 1 \\ 2  \\ 3 \end{bmatrix} \rightarrow \vec{y} = \begin{bmatrix} 1 & 1 \\ 1 & 2 \\ 1 & 3 \end{bmatrix} \begin{bmatrix} b \\ m \end{bmatrix}$$

Ou seja, vamos adicionar uma coluna com $1s$ no nosso vetor $x$ para simplificar as coisas. Vamos também chamar $b$ de $w_0$ e $m$ de $w_1$ (se tivéssemos várias variáveis, o peso $w_n$ multiplicaria a variável $n$). Assim nós também temos os vetor de pesos $\vec{w}$. Da mesma maneira, todas as nossas observações vão ser contidas em um vetor.

> info: O vetor $\vec{x_{obs}}$ virou a matrix $\bf{X_{obs}}$, que escrever como os $n$ vetores $\vec{x_{obs}^{(n)}}$ concatenados. Lembrando também que estamos falando de opeações de [matrizes](https://pt.wikipedia.org/wiki/Produto_de_matrizes), e que $\odot$ é a notação que eu escolhi para a multiplicação por elementos ([Hamadard product](https://en.wikipedia.org/wiki/Hadamard_product_(matrices))) de matrizes. Falando de matrizes, os índices das linhas e colunas aqui também vão de $0$ a $N-1$, onde $N$ são o número total de linhas ou colunas. Não é pra complicar, mas em [muitas das linguagens de programação eles denotam as matrizes assim (ao contrário da notação matemática comum)](https://en.wikipedia.org/wiki/Zero-based_numbering) - incluindo `python`.

$$\vec{x_{obs}^{(0)}} = \begin{bmatrix} 1 \\ 1 \\ 1 \end{bmatrix}, \vec{x_{obs}^{(1)}} = \begin{bmatrix} 1 \\ 2 \\ 3 \end{bmatrix}$$

$$\bf{X_{obs}} = \begin{bmatrix} 1 & 1 \\ 1 & 2 \\ 1 & 3 \end{bmatrix} = \begin{bmatrix} \vec{x_{obs}^{(0)}} & \vec{x_{obs}^{(1)}} \end{bmatrix}$$

$$\vec{w} =  \begin{bmatrix} w_0 \\ w_1 \end{bmatrix}$$

$$\vec{y_{obs}} = \begin{bmatrix} 2.5 \\ 5.5 \\ 6.5 \end{bmatrix}$$

$$\therefore \vec{y} = f(\vec{x}) = \vec{x} \vec{w}$$

$$w_0 \leftarrow w_0 - \alpha \cdot \frac{2}{N-1}\sum_{i=0}^{N-1}{((\vec{x} \vec{w} - \vec{y_{obs}})} \odot \vec{x_{obs}^{(0)}})_i$$

$$w_1 \leftarrow w_1 - \alpha \cdot \frac{2}{N-1}\sum_{i=0}^{N-1}{((\vec{x} \vec{w} - \vec{y_{obs}})} \odot \vec{x_{obs}^{(1)}})_i$$

E colocando todos os updates em uma equação só:

$$\therefore \vec{w_n} \leftarrow \vec{w_n} - \alpha \cdot \frac{2}{N-1}\sum_{i=0}^{N-1}{((\vec{x} \cdot \vec{w} - \vec{y_{obs}})} \odot \vec{x_{obs}^{(n)}})_i$$

> warning: Se você pesquisar, talvez você ache a última em formas ligeiramente diferentes. Quando isso acontecer, de uma olhada também nas definições dos vetores. É muito possível também que diferentes pessoas definam esses vetores de maneiras diferentes.

Eu sei que é muita coisa. Para. Pensa. Toma um café. E vê se faz sentido.

<div style="text-align:center"><img src="https://media.giphy.com/media/3o7TKTDn976rzVgky4/giphy.gif" width="30%"/></div>

#### Porque MSE?
Antes, a gente tinha escolhido o MSE (erro quadrático médio) pra descrever o erro da nossa função. Mas, porque usamos esse erro em particular?
Na verdade, essa escolha foi arbitrária. Existem outras funções de erro (também chamadas de função de custo/perda) que poderiam ser aplicadas. Inclusive, dependendo do problema algumas são mais interessantes que outras.

Por exemplo, poderiamos somar os valores absolutos dos erros (o chamado [erro médio absoluto](https://pt.qwe.wiki/wiki/Mean_absolute_error)) para definirmos os erros. Desde que essas funções sejam contínuas e que nós possamos definir uma derivada (já que a derivada define o passo em direção ao menor erro), vale tudo.

Algumas dessas funções são:
* [MSE](https://pt.qwe.wiki/wiki/Mean_squared_error)
* [MAE](https://pt.qwe.wiki/wiki/Mean_absolute_error)
* [Huber Loss](https://pt.qwe.wiki/wiki/Huber_loss)

#### Nomenclatura
As funções de erro são independentes do modelo de ML. Existem diferentes nomes na literatura para modelos em que tentamos encontrar a melhor linha para um grupo de pontos. Para essa implementação mais básica, você pode encontrar, entre outros,

* (Simple) linear regression
* Least squares regression
* Best fit curve

Além disso, muitas outras implementações mais sofisticadas tomam essa como base. Essas implementações mais sofisticadas levam em consideração o [sobreajuste ('overfitting', em inglês)](https://pt.wikipedia.org/wiki/Sobreajuste), ou então mais variáveis, ou até para aproximar uma função polinomial!

Bastante coisa! E de certa maneira, todas elas se baseiam na idéia de usar a derivada para encontrar o ponto com menor erro de uma função, então vale a pena investir o tempo para realmente entender o que está acontecendo aqui!

## Footnotes

{{ 'Veja o post sobre [gradient descent](todo_grad_desc) para mais detalhes.' | fndetail: 1 }}
{{ 'Mais precisamente, deveríamos estar falando de *massa* ao invés de *peso*. Mas no nosse approach isso não faz diferença.' | fndetail: 1 }}